# sofia-ml についての調査

ランク学習が利用できるという「sofia-ml」という機械学習ソフトウェアについて調査しました。

こちらを参考にいたしました。

https://code.google.com/archive/p/sofia-ml/

## (1) 概要

gcc 向けの C++ 言語により実装されております。

- Combined Regression and Ranking (CRR) という手法によるランク学習が使用できるとのことですが・・・

> The suite of fast incremental algorithms for machine learning (sofia-ml) can be used for training models for classification, regression, ranking, or combined regression and ranking.


- 実装のベースには SVM、ロジスティック回帰などが含まれるとのことです

> Suite of Fast Incremental Algorithms for Machine Learning. Includes methods for learning classification and ranking models, using Pegasos SVM, SGD-SVM, ROMMA, Passive-Aggressive Perceptron, Perceptron with Margins, and Logistic Regression.


- CRRアルゴリズムについての解説は以下URLに記載されています
 
　　http://www.eecs.tufts.edu/~dsculley/papers/combined-ranking-and-regression.pdf
 
 
- 「CRR」は、手法としては pointwise/pairwise に分類されるようです。

　　こちら (https://en.wikipedia.org/wiki/Learning_to_rank#Pointwise_approach) の「List of methods」をご参照
 

- 入力データ（教師データ）形式

　　デモデータを見る限りでは<a href="01-SVMRank-outline.ipynb"><b>SVMRank</b></a>で使用したものと同様ですが、qid は有りませんでした。
  
　　2017/05/04時点で詳細は不明ですが、おそらく学習手法によって、与えるデータのフォーマットが異なるものと推測しています。

## (2) 環境準備

### (2-1) リンク先からファイルを取得

下記リンクからソースコードのアーカイブを取得します。

https://storage.googleapis.com/google-code-archive-source/v2/code.google.com/sofia-ml/source-archive.zip

取得後、下記リンクの「Quick Start」手順に従いビルドします。

https://code.google.com/archive/p/sofia-ml/

```
MacBookPro-makmorit-jp:sofia-ml makmorit$ pwd
/Users/makmorit/GitHub/donusagi-bot/learning/prototype/rank_learning/sofia-ml/trunk/sofia-ml
MacBookPro-makmorit-jp:sofia-ml makmorit$ ls -al
total 80
drwxr-xr-x@ 11 makmorit  staff    374 May  4 19:25 .
drwxr-xr-x@  5 makmorit  staff    170 May  4 19:25 ..
-rw-r--r--@  1 makmorit  staff   6148 May  4 20:29 .DS_Store
drwxr-xr-x@  8 makmorit  staff    272 Feb 17  2016 .svn
-rw-r--r--@  1 makmorit  staff  11358 Feb 17  2016 COPYING
-rw-r--r--@  1 makmorit  staff  12576 Feb 17  2016 README
drwxr-xr-x@ 13 makmorit  staff    442 Feb 17  2016 cluster-src
drwxr-xr-x@  5 makmorit  staff    170 Feb 17  2016 demo
-rwxr-xr-x@  1 makmorit  staff   3838 Feb 17  2016 eval.pl
drwxr-xr-x@  4 makmorit  staff    136 Feb 17  2016 papers
drwxr-xr-x@ 25 makmorit  staff    850 Feb 17  2016 src
MacBookPro-makmorit-jp:sofia-ml makmorit$ cd src
MacBookPro-makmorit-jp:src makmorit$ make
g++ -O3 -lm -Wall -o sofia-ml sofia-ml.cc sofia-ml-methods.cc sf-weight-vector.cc sf-sparse-vector.cc sf-data-set.cc sf-hash-weight-vector.cc sf-hash-inline.cc
cp sofia-ml ..
MacBookPro-makmorit-jp:src makmorit$ cd ..
MacBookPro-makmorit-jp:sofia-ml makmorit$ ls -al
total 368
drwxr-xr-x@ 12 makmorit  staff     408 May  4 20:31 .
drwxr-xr-x@  5 makmorit  staff     170 May  4 19:25 ..
-rw-r--r--@  1 makmorit  staff    6148 May  4 20:29 .DS_Store
drwxr-xr-x@  8 makmorit  staff     272 Feb 17  2016 .svn
-rw-r--r--@  1 makmorit  staff   11358 Feb 17  2016 COPYING
-rw-r--r--@  1 makmorit  staff   12576 Feb 17  2016 README
drwxr-xr-x@ 13 makmorit  staff     442 Feb 17  2016 cluster-src
drwxr-xr-x@  5 makmorit  staff     170 Feb 17  2016 demo
-rwxr-xr-x@  1 makmorit  staff    3838 Feb 17  2016 eval.pl
drwxr-xr-x@  4 makmorit  staff     136 Feb 17  2016 papers
-rwxr-xr-x   1 makmorit  staff  144384 May  4 20:31 sofia-ml  <---これが実行可能ファイルのようです。
drwxr-xr-x@ 26 makmorit  staff     884 May  4 20:31 src
MacBookPro-makmorit-jp:sofia-ml makmorit$ 
```

### (2-2) 環境のベリファイ

実行可能ファイル sofia-ml を引数なしで実行させます。

コマンドラインオプションの一覧が表示されればOKです。

```
MacBookPro-makmorit-jp:sofia-ml makmorit$ ./sofia-ml
Command line flag options: 
      --buffer_mb           Size of buffer to use in reading/writing to files, in MB.
    Default: 40

      --dimensionality      Index value of largest feature index in training data set. 
    Default: 2^17 = 131072

      --eta_type            Type of update for learning rate to use.
    Options are: basic, pegasos, constant (0.02).
    Default: pegasos

      --hash_mask_bits      When set to a non-zero value, causes the use of a hashed weight vector
    with hashed cross product features.  The size of the hash table is set
    to 2^--hash_mask_bits.  The same value of this flag must be used for
    testing and training.
    Default value of 0 shows that hash cross products are not used.

      --iterations          Number of stochastic gradient steps to take.
    Default: 100000

      --lambda              Value of lambda for svm regularization.
    Default: 0.1

      --learner_type        Type of learner to use.
    Options are: pegasos, passive-aggressive, margin-perceptron, romma, sgd-svm, least-mean-squares, logreg, and logreg-pegasos.
    Default: pegasos

      --loop_type           Type of loop to use for training, controlling how examples are selected.
    Options are: stochastic, balanced-stochastic, roc, rank, query-norm-rank, combined-ranking, combined-roc
    Default: stochastic

      --model_in            Read in a model from this file.

      --model_out           Write the model to this file.

      --no_bias_term        When set, causes a bias term x_0 to be set to 0 for every 
    feature vector loaded from files, rather than the default of x_0 = 1.
    This is equivalent to forcing a decision threshold of exactly 0 to be used.
    Same setting of this flag should be used for training and testing. Note that
    this flag as no effect for rank and roc optimzation.
    Default: not set.

      --passive_aggressive_c  Maximum size of any step taken in a single passive-aggressive update.

      --passive_aggressive_lambda  Lambda for pegasos-style projection for passive-aggressive update.
    When set to 0 (default) no projection is performed.

      --perceptron_margin_size  Width of margin for perceptron with margins.
    Default of 1 is equivalent to un-regularized SVM-loss.

      --prediction_type     Type of predictions to compute on test data.
    Options are: linear, logistic
    Use linear for SVM predictions.
    Default: linear

      --random_seed         When set to non-zero value, use this seed instead of seed from system clock.
    This can be useful for parameter tuning in cross-validation, as setting 
    a seed by hand forces examples to be sampled in the same order.  However
    for actual training/test, this should never be used.
    Default: 0

      --rank_step_probability  Probability that we will take a rank step (as opposed to a  
    classification step) in a combined-ranking or     combined-roc loop.
    Default: 0.5

      --results_file        File to which to write predictions.

      --test_file           File to be used for testing.

      --training_file       File to be used for training.

      --training_objective  Compute value of objective function on training data, after training.
    Default is not to do this.
```

### (2-3) テストデータの準備

デモ用のデータがあらかじめ用意されています。（ただしこれは２値判定 SVM 用のものっぽいです）

今回は稼動確認目的のため、別段編集しないで動作させてみます。

```
MacBookPro-makmorit-jp:demo makmorit$ pwd
/Users/makmorit/GitHub/donusagi-bot/learning/prototype/rank_learning/sofia-ml/trunk/sofia-ml/demo
MacBookPro-makmorit-jp:demo makmorit$ ls -al
total 4656
drwxr-xr-x@  5 makmorit  staff      170 Feb 17  2016 .
drwxr-xr-x@ 12 makmorit  staff      408 May  4 20:31 ..
drwxr-xr-x@  8 makmorit  staff      272 Feb 17  2016 .svn
-rwxr-xr-x@  1 makmorit  staff  1196048 Feb 17  2016 demo.test
-rw-r--r--@  1 makmorit  staff  1180410 Feb 17  2016 demo.train
MacBookPro-makmorit-jp:demo makmorit$ head -2 demo.train
-1 1:1 7:1 8:1 10:1 12:1 13:1 14:1 15:1 16:1 17:1 19:1 20:1 21:1 22:1 24:1 28:1 29:1 30:1 31:1 32:1 33:1 35:1 38:1 39:1 42:1 43:1 74:1 76:1 78:1 79:1 86:1 90:1 91:1 92:1 122:1 132:1 145:1 147:1 160:1 162:1 170:1 177:1 205:1 206:1 207:1 208:1 209:1 210:1 211:1 212:1 213:1 214:1 215:1 216:1 217:1 218:1 219:1 220:1 221:1 222:1 223:1 224:1 225:1 226:1 227:1 228:1 229:1 230:1 231:1 232:1 233:1 234:1 235:1 236:1 237:1 238:1 239:1 240:1 241:1 242:1 243:1 244:1 245:1 246:1 247:1 248:1 249:1 250:1 251:1 252:1 253:1 254:1 255:1 256:1 257:1 258:1 259:1 260:1 261:1 262:1 263:1 264:1 265:1 266:1 267:1 268:1 269:1 270:1 271:1 272:1 273:1 274:1 275:1 276:1 277:1 278:1 279:1 280:1 281:1 282:1 283:1 284:1 285:1 286:1 287:1 288:1 289:1 290:1 291:1 292:1 293:1 294:1 295:1 296:1 297:1 298:1 299:1 300:1 301:1 302:1 303:1 304:1 305:1 306:1 307:1 308:1 309:1 310:1 311:1 312:1 313:1 314:1 315:1 316:1 317:1 318:1 319:1 320:1 321:1 322:1 323:1 324:1 325:1 326:1 327:1 328:1 329:1 330:1 331:1 332:1 333:1 334:1 335:1 336:1 337:1 338:1 339:1 340:1 
1 1:1 2:1 3:1 4:1 5:1 6:1 7:1 8:1 9:1 10:1 11:1 12:1 13:1 14:1 15:1 16:1 17:1 18:1 19:1 20:1 21:1 22:1 23:1 24:1 25:1 26:1 27:1 28:1 29:1 30:1 31:1 32:1 33:1 34:1 35:1 36:1 37:1 38:1 39:1 40:1 41:1 42:1 43:1 44:1 45:1 46:1 47:1 48:1 49:1 50:1 51:1 52:1 53:1 54:1 55:1 56:1 57:1 58:1 59:1 60:1 61:1 62:1 63:1 64:1 65:1 66:1 67:1 68:1 69:1 70:1 71:1 72:1 73:1 74:1 75:1 76:1 77:1 78:1 79:1 80:1 81:1 82:1 83:1 84:1 85:1 86:1 87:1 88:1 89:1 90:1 91:1 92:1 93:1 94:1 95:1 96:1 97:1 98:1 99:1 100:1 101:1 102:1 103:1 104:1 105:1 106:1 107:1 108:1 109:1 110:1 111:1 112:1 113:1 114:1 115:1 116:1 117:1 118:1 119:1 120:1 121:1 122:1 123:1 124:1 125:1 126:1 127:1 128:1 129:1 130:1 131:1 132:1 133:1 134:1 135:1 136:1 137:1 138:1 139:1 140:1 141:1 142:1 143:1 144:1 145:1 146:1 147:1 148:1 149:1 150:1 151:1 152:1 153:1 154:1 155:1 156:1 157:1 158:1 159:1 160:1 161:1 162:1 163:1 164:1 165:1 166:1 167:1 168:1 169:1 170:1 171:1 172:1 173:1 174:1 175:1 176:1 177:1 178:1 179:1 180:1 181:1 182:1 183:1 184:1 185:1 186:1 187:1 188:1 189:1 190:1 191:1 192:1 193:1 194:1 195:1 196:1 197:1 198:1 199:1 200:1 201:1 202:1 203:1 204:1 
MacBookPro-makmorit-jp:demo makmorit$ head -2 demo.test
1 7:1 12:1 13:1 15:1 16:1 17:1 20:1 21:1 22:1 24:1 28:1 29:1 43:1 52:1 74:1 76:1 78:1 79:1 82:1 86:1 90:1 91:1 92:1 95:1 106:1 122:1 132:1 145:1 147:1 158:1 160:1 162:1 177:1 182:1 188:1 300:1 342:1 409:1 410:1 423:1 434:1 456:1 465:1 473:1 484:1 488:1 530:1 548:1 569:1 577:1 649:1 654:1 657:1 697:1 860:1 894:1 1069:1 1070:1 1091:1 1343:1 1425:1 1462:1 1600:1 1622:1 1623:1 1625:1 1626:1 1627:1 1628:1 1629:1 1631:1 1632:1 1633:1 1634:1 1635:1 1637:1 1639:1 1640:1 1641:1 1643:1 1645:1 1646:1 1647:1 1648:1 1652:1 1655:1 1657:1 1658:1 1661:1 1662:1 1664:1 1977:1 2025:1 2364:1 2485:1 2947:1 3359:1 3898:1 3951:1 3955:1 4043:1 4044:1 4118:1 4740:1 5009:1 5017:1 5068:1 5082:1 5087:1 5535:1 5542:1 5742:1 7457:1 7671:1 7676:1 7682:1 9159:1 9160:1 9778:1 10135:1 11394:1 12742:1 13092:1 13852:1 14136:1 16866:1 16877:1 17045:1 17549:1 17855:1 17935:1 18166:1 20923:1 21025:1 23370:1 23381:1 25904:1 26826:1 31742:1 35064:1 36567:1 37133:1 38330:1 38334:1 42109:1 44119:1 48503:1 52640:1 53081:1 54286:1 54289:1 54290:1 54292:1 54294:1 54295:1 54300:1 54303:1 54304:1 76324:1 81317:1 81318:1 81319:1 81320:1 81321:1 81322:1 81323:1 81324:1 81325:1 81326:1 81327:1 81328:1 81329:1 81330:1 81331:1 81332:1 81333:1 81334:1 81335:1 81336:1 81337:1 81338:1 81339:1 81340:1 81341:1 81342:1 81343:1 81344:1 81345:1 81346:1 81347:1 
1 1:1 7:1 8:1 10:1 11:1 12:1 13:1 14:1 15:1 16:1 17:1 18:1 19:1 20:1 21:1 22:1 24:1 25:1 28:1 29:1 33:1 34:1 35:1 38:1 39:1 65:1 76:1 78:1 79:1 86:1 90:1 91:1 92:1 93:1 94:1 95:1 96:1 97:1 122:1 132:1 149:1 162:1 206:1 207:1 215:1 226:1 268:1 307:1 354:1 436:1 449:1 473:1 479:1 504:1 537:1 649:1 657:1 752:1 980:1 1019:1 1020:1 1542:1 1681:1 1831:1 1871:1 1876:1 1877:1 1878:1 1880:1 1882:1 1883:1 1884:1 1885:1 1886:1 1889:1 1891:1 1892:1 1893:1 1894:1 2128:1 2363:1 2525:1 3404:1 3430:1 3431:1 3432:1 3434:1 3435:1 3436:1 3439:1 3556:1 3948:1 3954:1 4845:1 5158:1 5245:1 6199:1 7344:1 7356:1 8961:1 9435:1 9625:1 11835:1 12108:1 13019:1 13850:1 13929:1 14848:1 16918:1 18610:1 20455:1 22529:1 23005:1 23247:1 25515:1 32110:1 36983:1 45054:1 45262:1 46876:1 46893:1 46894:1 46899:1 46901:1 46903:1 46911:1 46912:1 54945:1 70095:1 81348:1 81349:1 81350:1 81351:1 81352:1 81353:1 81354:1 81355:1 81356:1 81357:1 81358:1 81359:1 81360:1 81361:1 81362:1 81363:1 81364:1 81365:1 81366:1 81367:1 81368:1 81369:1 81370:1 81371:1 81372:1 81373:1 81374:1 81375:1 81376:1 81377:1 81378:1 81379:1 81380:1 81381:1 81382:1 81383:1 81384:1 81385:1 81386:1 81387:1 
MacBookPro-makmorit-jp:demo makmorit$```

## (3) 動作確認

### (3-1) 学習処理

学習セットファイル demo/demo.train を引数として学習処理を行うと、demo/model というファイルにモデルが生成されます。

- コマンド例：
> ./sofia-ml --learner_type pegasos --loop_type stochastic --lambda 0.1 --iterations 100000 --dimensionality 150000 --training_file demo/demo.train --model_out demo/model


```
MacBookPro-makmorit-jp:sofia-ml makmorit$ ./sofia-ml --learner_type pegasos --loop_type stochastic --lambda 0.1 --iterations 100000 --dimensionality 150000 --training_file demo/demo.train --model_out demo/model
Reading training data from: demo/demo.train
Time to read training data: 0.020424
Time to complete training: 0.030156
Writing model to: demo/model
   Done.
MacBookPro-makmorit-jp:sofia-ml makmorit$ ls -al demo/model
-rw-r--r--  1 makmorit  staff  478254 May  4 20:48 demo/model
MacBookPro-makmorit-jp:sofia-ml makmorit$ 
```

モデルファイルの中身はサポートベクトルとおぼしきデータの羅列となっておりました。

```
MacBookPro-makmorit-jp:sofia-ml makmorit$ cat demo/model
-0.0260576 0.0527387 0 0.00985612 -0.048184 0.0321594 0 -0.0373082 -0.0624532 0 0.0207028 0.111987 0.0247773 0.00794251 -0.0227479 -0.0383746 0.0191478 0.0168772 0.0602054 -0.0235973 0.00728161 0.00353397 0.0271773 0 -0.00146194 0.0611017 -0.0082263 -0.0034 0.000898209 -0.00414554 -0.0324246 -0.0702588 -0.0554568 -0.0189478 0.0611017 -0.0189478 0.0169548 0 -0.0193444 -0.00449018 0 0 -0.0524779 -0.0328859 0 0.0369893 0.0120037 0.0485294 0 0 0 0 0.140143 0.0297357 0.0232356 0.0242024 0.0116851 0.0168041 0.0225356 0.0175763 0 0 0.0161426 0 0 0.0479388 0 0 -0.0004 0 0 0.0055 0 0 0.0591774 0 0.0279773 0 -0.0141343 -0.0176575 0.00299231 -0.0105 0.0147099 -0.0211247 0.0188168 0 0.00752237 -0.00112812 0 0.0255763 0.0520301 0.03683 0.048607 0.0881782 0.0373659 0.0686764 0.0404822 0.0386361 0.00585963 -0.0041 0 0.00451862 0.0007 0 0.0015 0 -0.0021942 0.0015 0 0 0 0.0155956 0.0032 0 0 0.0199686 0.0006 0 0 0.0157718 0 0 -0.0202148 0.0045 0 0 0.0011 0 0 0 -0.0027 0 -0.0195322 0.00416773 0 -0.0054 0.0126 0.0054 0 0.00620983 0 0 0.0069 0 -0.0139846 -0.00412637 0 -0.00255961 0.0029 -0.0164 -0.00298126 0.0004 0.0133956 0 0 0 0 0.0102 0.0444628 0.00732971 0.0574808 0.000997336 -0.0421308 0.0001 0 0.0292428 0 0.0255068 0 0 -0.0189981 0 0 0 0.0472391 -0.0178768 0 -0.0134707 0.0023 0 0 0 0.0081416 0 0.0239001 0 0 0 0.0827978 0.0076202 0.0045 0.0019 0 0.00970579 0.0128643 0 0 0.0009 -0.0111568 0 0 0 0 0 0.0021 -0.0116101 -0.000487625 -0.015182 -0.0115711 0.00159996 -0.0116101 0.0270623 -0.0166958 （以下長いので省略します）
```

### (3-2) 予測処理

テストデータファイル demo/demo.test を引数として予測処理処理を行うと、results.txt に予測結果が格納されます。

- コマンド例

> ./sofia-ml --model_in demo/model --test_file demo/demo.test --results_file demo/results.txt


```
MacBookPro-makmorit-jp:sofia-ml makmorit$ ./sofia-ml --model_in demo/model --test_file demo/demo.test --results_file demo/results.txt
Reading model from: demo/model
   Done.
Reading test data from: demo/demo.test
Time to read test data: 0.019281
Time to make test prediction results: 0.000396
Writing test results to: demo/results.txt
   Done.
MacBookPro-makmorit-jp:sofia-ml makmorit$ 
```

### (3-3) 予測結果を参照

results.txt に、スコアと予測結果が、タブ区切りで出力されます。

（ちなみに、テストデータのレコードと同じ並びになっているとのことです。

　予測対象のテストデータが１件であれば、出力も１件になるかと存じます）

今回使用したテストデータは２値判定用なので、ラベルは 1 or -1 となっていますが、通常はラベルが入るものと考えられます。

```
MacBookPro-makmorit-jp:sofia-ml makmorit$ head -5 demo/results.txt
1.03036	1
1.10212	1
-1.23624	-1
-1.11984	-1
-1.40745	-1
MacBookPro-makmorit-jp:sofia-ml makmorit$ 
```

### (3-4) モデルの評価

評価用の perl スクリプトが用意されていますので、使用してみます。

（2017/05/03現在、詳細については不明）

今回使用したテストデータには、（なぜか）正解データが含まれている為、それをもとにAccuracyを算定している様子です。

本来は学習セット（上記の demo/demo.train）またはその一部を用いて予測処理を行い、その結果でもって評価を行う運用になるかと存じます。

```
MacBookPro-makmorit-jp:sofia-ml makmorit$ perl eval.pl demo/results.txt

Results for demo/results.txt: 

Accuracy  0.9880  (using threshold 0.00) (988/1000)
Precision 0.9719  (using threshold 0.00) (311/320)
Recall    0.9904  (using threshold 0.00) (311/314)
ROC area: 0.999387 

Total of 1000 trials. 

MacBookPro-makmorit-jp:sofia-ml makmorit$ 
```

## (4) ランク学習処理の確認

sofia-ml を使用してランク学習／予測の動作確認を行いました。

結論から申し上げると、前述デモのようなSVMによるクラス分類はともかく、ランク学習には使えそうにないという印象を受けました。

### (4-1) テストデータの準備

Microsoft のランク学習用ベンチマークデータ（MSLR）をダウンロードし、テストデータを作成しました。

こちらからダウンロードしています

https://www.microsoft.com/en-us/research/project/mslr/ <--- Microsoft Learning to Rank Datasetsの説明ページ

https://1drv.ms/u/s!AtsMfWUz5l8nbOIoJ6Ks0bEMp78 <--- このリンクをクリックする

```
MacBookPro-makmorit-jp:MofMofInc makmorit$ ls -alR MSLR-WEB10K
total 16
drwx------@  8 makmorit  staff   272 May  4 20:09 .
drwxr-xr-x  18 makmorit  staff   612 May  5 10:47 ..
-rw-r--r--@  1 makmorit  staff  6148 May  5 12:07 .DS_Store
drwxr-xr-x@  5 makmorit  staff   170 Apr 28  2010 Fold1
drwxr-xr-x@  5 makmorit  staff   170 Apr 28  2010 Fold2
drwxr-xr-x@  5 makmorit  staff   170 Apr 28  2010 Fold3
drwxr-xr-x@  5 makmorit  staff   170 Apr 28  2010 Fold4
drwxr-xr-x@  5 makmorit  staff   170 Apr 28  2010 Fold5

MSLR-WEB10K/Fold1:
total 2714904
drwxr-xr-x@ 5 makmorit  staff        170 Apr 28  2010 .
drwx------@ 8 makmorit  staff        272 May  4 20:09 ..
-rwxr-xr-x@ 1 makmorit  staff  279254654 Apr 30  2010 test.txt
-rwxr-xr-x@ 1 makmorit  staff  838011150 Apr 30  2010 train.txt
-rwxr-xr-x@ 1 makmorit  staff  272760099 Apr 30  2010 vali.txt

MSLR-WEB10K/Fold2:
total 2714904
drwxr-xr-x@ 5 makmorit  staff        170 Apr 28  2010 .
drwx------@ 8 makmorit  staff        272 May  4 20:09 ..
-rwxr-xr-x@ 1 makmorit  staff  280714022 Apr 30  2010 test.txt
-rwxr-xr-x@ 1 makmorit  staff  830057227 Apr 30  2010 train.txt
-rwxr-xr-x@ 1 makmorit  staff  279254654 Apr 30  2010 vali.txt

MSLR-WEB10K/Fold3:
total 2714904
drwxr-xr-x@ 5 makmorit  staff        170 Apr 28  2010 .
drwx------@ 8 makmorit  staff        272 May  4 20:09 ..
-rwxr-xr-x@ 1 makmorit  staff  276606739 Apr 30  2010 test.txt
-rwxr-xr-x@ 1 makmorit  staff  832705142 Apr 30  2010 train.txt
-rwxr-xr-x@ 1 makmorit  staff  280714022 Apr 30  2010 vali.txt

MSLR-WEB10K/Fold4:
total 2714896
drwxr-xr-x@ 5 makmorit  staff        170 Apr 28  2010 .
drwx------@ 8 makmorit  staff        272 May  4 20:09 ..
-rwxr-xr-x@ 1 makmorit  staff  280690389 Apr 30  2010 test.txt
-rwxr-xr-x@ 1 makmorit  staff  832728775 Apr 30  2010 train.txt
-rwxr-xr-x@ 1 makmorit  staff  276606739 Apr 30  2010 vali.txt

MSLR-WEB10K/Fold5:
total 2714904
drwxr-xr-x@ 5 makmorit  staff        170 Apr 28  2010 .
drwx------@ 8 makmorit  staff        272 May  4 20:09 ..
-rwxr-xr-x@ 1 makmorit  staff  272760099 Apr 30  2010 test.txt
-rwxr-xr-x@ 1 makmorit  staff  836575415 Apr 30  2010 train.txt
-rwxr-xr-x@ 1 makmorit  staff  280690389 Apr 30  2010 vali.txt
```

### (4-2) 学習処理

MSLRのデータセットは非常に大きいので、教師データ（train.txt）と正解ラベル付きのテストデータ（vali.txt）を、それぞれ10,000件／2,500件ずつ切り出して使用します。

その後、任意のエディターで「CR+LF」を「LF」だけに変換しておきます。

（CRコードが入っていると、なぜかsofia-mlがエラー「Segmentation fault: 11」を発生させダウンしてしまいます）

```
MacBookPro-makmorit-jp:sofia-ml makmorit$ pwd
/Users/makmorit/GitHub/donusagi-bot/learning/prototype/rank_learning/sofia-ml/trunk/sofia-ml
MacBookPro-makmorit-jp:sofia-ml makmorit$ head -10000 ~/Documents/Development/MofMofInc/MSLR-WEB10K/Fold1/train.txt > data/train.txt
MacBookPro-makmorit-jp:sofia-ml makmorit$ head -2500  ~/Documents/Development/MofMofInc/MSLR-WEB10K/Fold1/vali.txt > data/vali.txt
```

ちなみに学習データのフォーマットはSVMRankなどに使用するものと一緒で、概ね下記のようになります。

>&lt;class-label&gt;△qid:&lt;optional-query-id&gt;△&lt;feature-id&gt;:&lt;feature-value&gt;

（△は半角スペース文字）

class-labelとあるのはラベルを意味するそうですが、ランク学習の場合はランクを示す適合度を指定します。

すなわち、ラベルの値が大きいほど適合度が高い＝ランキングが上位、ということになるかと存じます。

>The class label for test data is required but not used; it's okay to put in a dummy placeholder value such as 0 for test data. For binary-class classification problems, the training labels should be 1 or -1. For ranking problems, the labels may be any numeric value, with higher values being judged as "more preferred".

```
MacBookPro-makmorit-jp:sofia-ml makmorit$ head -5 data/train.txt
2 qid:1 1:3 2:3 3:0 4:0 5:3 6:1 7:1 8:0 9:0 10:1（以下略）
2 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1（以下略）
0 qid:1 1:3 2:0 3:2 4:0 5:3 6:1 7:0 8:0.666667 9:0 10:1（以下略）
2 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1（以下略）
1 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1（以下略）
```

学習セットファイル data/train.txt を引数として学習処理を行うと、data/model.out というファイルにモデルが生成されます。

- コマンド例（デフォルトで指定される引数設定は省略します）

> ./sofia-ml --loop_type combined-ranking --training_file data/train.txt --model_out data/model.out

- loop_type という引数に combined-ranking を指定します。CRRアルゴリズムを使用するようです。

> * combined-ranking Performs CRR algorithm for combined regression and ranking. Alternates between pairwise rank-based steps and standard stochastic gradient steps on single examples.

```
MacBookPro-makmorit-jp:sofia-ml makmorit$ ./sofia-ml --loop_type combined-ranking --training_file data/train.txt --model_out data/model.out
Reading training data from: data/train.txt
Time to read training data: 0.189115
Time to complete training: 0.135738
Writing model to: data/model.out
   Done.
```

### (4-3) 予測処理

テストデータファイル data/vali.txt を引数として予測処理処理を行うと、data/vali-result.txt に予測結果が格納されます。

- コマンド例

> ./sofia-ml --model_in data/model.out --test_file data/vali.txt --results_file data/vali-result.txt


```
MacBookPro-makmorit-jp:sofia-ml makmorit$ ./sofia-ml --model_in data/model.out --test_file data/vali.txt --results_file data/vali-result.txt
Reading model from: data/model.out
   Done.
Reading test data from: data/vali.txt
Time to read test data: 0.047355
Time to make test prediction results: 0.000539
Writing test results to: data/vali-result.txt
   Done.
```

### (4-4) 予測結果を参照

results.txt に、スコアとラベルが、タブ区切りで出力されます。

ちなみに、予測結果ファイルのレコードは、テストデータのレコードと同じ並びになっているとのことです。

また、予測結果ファイルのラベルは、テストデータに格納されたものと同値です。

>--results_file file 
>
>File to which to write predictions, when --test_file is used. Results for each line are in the format &lt;prediction&gt;\t&lt;label from test file&gt;
>
>and correspond line-by-line with the examples form the --test_file.


```
MacBookPro-makmorit-jp:sofia-ml makmorit$ head -20 data/vali-result.txt
84913    0 <---ラベル０のスコアが、ラベル２のスコアを上回っている
6095.63    0
87700    1
21552.3    0
72224.2    1 <---ラベル１のスコアが、ラベル２のスコアを上回っている
61089.2    2 <---ラベル２のスコア（すなわちここに表示されているものの中で、最もランクが高いはず）
56598.7    1
61460.9    1
61529.4    0
106045    0
63855.6    0
13307.6    1
67008.9    0
74871.7    1
40840.4    0
56024.3    0
87642.9    0 <---ラベル０のスコアが、ラベル２のスコアを上回っている
74126.9    1
87030.8    0 <---ラベル０のスコアが、ラベル２のスコアを上回っている
18443.2    1
```

上記、予測結果ファイルの内容を見た限りでは、ラベル０（＝ランクが低い）のスコアが、ラベル２（＝ランクが高い）のスコアを上回っているなど、ランキング・スコアを示しているとは思えない結果となっておりました。

### (4-5) モデルの評価

評価用として提供されていた perl スクリプトは、（デモ用の）２値判定用SVMの結果評価に特化したもののようで、ランク学習には使用できませんでした。

（無理に実行させようとすると、Illegal division by zero のエラーでダウンします）

```
MacBookPro-makmorit-jp:sofia-ml makmorit$ perl eval.pl data/vali-result.txt
Illegal division by zero at eval.pl line 81.
```